In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import jieba
import re
import pdb
#from gensim.models import word2vec

In [2]:
# 读取数据
train = pd.read_csv('data/train.tsv', sep='\t',header=0)
test = pd.read_csv('data/test.tsv', sep='\t',header=0)
dev = pd.read_csv('data/dev.tsv', sep='\t',header=0)
train_data = pd.concat([train, dev, test])
print(train.shape)
print(dev.shape)
print(test.shape)
print(train_data.shape)
train_data = train_data.reset_index()

(6000, 4)
(1240, 4)
(2000, 4)
(9240, 4)


In [3]:
# 根据关键词将每句话分成left和right
left_input=[]
right_input=[]
for i in range(train_data.shape[0]):
    # questions（namely，实体名字）为分界线进行划分
    left,target,right = train_data['sentence'][i].partition(train_data['question'][i])
    left_input.append(left+target)
    right_input.append(target+right)


In [4]:
left_input[0]

'也就是说，齐星集团'

In [5]:
right_input[0]

'齐星集团等二十七家公司已经处于资不抵债状态。而管理人自接受法院指定以来，招募重整投资人也成了头等要务。'

In [6]:
train_data = train_data.drop(['index'],axis=1)
train_data = train_data.drop(['question'],axis=1)
train_data = train_data.reset_index()

In [7]:
# 建立停用词表
stopwords = [line.strip() for line in open('hit_stop_word.txt', 'r', encoding='utf-8').readlines()]
others = ['﹏', '\\', '◦', '˙', 'っ', '˘', 'ڡ', '˘', 'ς', '´', 'ﾉ']

from jieba import cut
from tensorflow.contrib import learn
import numpy as np
#pdb.set_trace()
DOCUMENTS = train_data['sentence'].values
# 自定义的tokener函数
def chinese_tokenizer(comment_list):
    for comment in comment_list:
        comment = re.sub('[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+', " ", comment)
        temp = list(jieba.cut(comment))
        temp_list = []
        for word in temp:
            # 去除停用词
            if word not in stopwords:
                if word not in others:
                    # 清除空格
                    if word != ' ':
                        temp_list.append(word)
        
        yield temp_list

# 使用tf的VocabularyProcessor自定义tokenizer函数进行分词
#建立一个分词器
vocab = learn.preprocessing.VocabularyProcessor(100, 0, tokenizer_fn=chinese_tokenizer)
#进行分词 将left和right都投入进去
x = list(vocab.fit_transform(left_input+right_input))
# 结果转换成list
features=[i.tolist() for i in x]
features = np.array(features)
print(features.shape)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.667 seconds.
Prefix dict has been built succesfully.


(18480, 100)


In [8]:
# 获得字典
word_index = vocab.vocabulary_._mapping

In [9]:
# 读取腾讯训练好的vector
f = open('/home/zhuqinglin/Jupyter/Tencent_AILab_ChineseEmbedding.txt',encoding = 'utf-8')
embeddings_index = {}
#pdb.set_trace()
for line in f:
    values = line.split()
    word = values[0] #提取word
    try:
        coefs = np.asarray(values[1:], dtype='float32')#提取vector
    except ValueError:
        print(values)
    embeddings_index[word] = coefs # 保存下来
f.close()
len(embeddings_index)

['中共中央', '国务院关于完善产权保护制度依法保护产权的意见', '0.141571', '-0.006408', '-0.813869', '0.364291', '0.522222', '0.095206', '-0.182812', '0.382395', '-0.153602', '-0.054792', '-0.207183', '-0.232800', '0.083588', '-0.138412', '-0.209798', '-0.266059', '-0.045677', '-0.103303', '0.541334', '-0.047070', '-0.283453', '0.361866', '0.048522', '0.506735', '0.395017', '0.516819', '0.008693', '0.558315', '-0.284115', '0.183513', '-0.419605', '-0.019989', '0.329071', '0.270945', '-0.650149', '-0.313131', '0.010436', '-0.339008', '-0.605940', '0.048549', '0.150862', '0.087893', '0.817539', '0.066148', '-0.444597', '0.124834', '-0.441955', '-0.379341', '0.302012', '-0.187727', '0.074427', '-0.058798', '-0.039492', '0.510873', '0.394675', '0.095644', '0.037183', '0.413851', '-0.482928', '-0.345242', '0.312569', '0.544982', '0.380840', '0.501457', '-0.139450', '-0.051441', '-0.383388', '-0.017092', '0.023642', '0.085509', '-0.206751', '0.062732', '0.207283', '-0.411276', '0.121205', '0.018632', '0.226613', '0.145

['共担时代责任', '共促全球发展', '0.357690', '0.097080', '0.179483', '0.402549', '0.593071', '0.178165', '0.398317', '0.396116', '0.226220', '0.820646', '0.378178', '-0.277740', '-0.547582', '0.099789', '0.014076', '0.202386', '-0.003464', '-0.086524', '-0.042962', '-0.331683', '0.157536', '0.154902', '0.012900', '0.471988', '0.177722', '0.436315', '0.220469', '0.093384', '0.162547', '-0.009418', '-0.214824', '0.020734', '0.255046', '0.470425', '-0.462638', '-0.015007', '0.304401', '0.262299', '-0.172130', '0.115146', '0.112616', '0.416724', '0.434991', '-0.126485', '-0.290469', '0.509502', '0.065550', '-0.384226', '0.064767', '0.337985', '0.312837', '0.198498', '-0.519673', '0.026272', '0.741239', '0.113647', '-0.013332', '0.660608', '-0.194515', '0.270333', '0.494415', '0.102755', '0.513382', '0.055199', '0.535765', '0.112021', '0.017534', '0.198559', '0.195787', '0.209493', '0.115608', '-0.299623', '0.421322', '-0.339797', '0.138541', '-0.147917', '0.200387', '-0.278653', '-0.063651', '0.555896

['贾夫人仙逝扬州城', '冷子兴演说荣国府', '0.435553', '0.001867', '0.366112', '-0.041674', '0.337835', '0.016730', '0.491745', '0.100708', '-0.257096', '-0.016911', '-0.075895', '0.102955', '0.067547', '-0.401133', '0.122182', '0.134846', '-0.436403', '0.098773', '0.450619', '-0.047825', '-0.341748', '0.584128', '-0.022972', '0.090945', '-0.124144', '-0.038321', '-0.233966', '0.131769', '-0.097448', '-0.263433', '-0.251568', '0.074292', '-0.280752', '-0.122769', '-0.355704', '-0.328443', '-0.118153', '-0.058597', '-0.522994', '0.793508', '0.469465', '0.339674', '0.564040', '-0.370377', '0.177015', '-0.212880', '-0.294062', '-0.197852', '0.100556', '-0.073668', '0.315706', '0.014535', '0.009390', '0.180290', '0.049778', '-0.358335', '0.097179', '0.330571', '-0.032581', '0.238276', '0.272048', '0.090963', '0.482459', '0.215860', '0.339282', '0.130900', '0.094928', '0.485619', '-0.300908', '0.208348', '-0.427007', '0.261190', '-0.024019', '0.151834', '0.039447', '0.412389', '0.067662', '0.211842', '-0.075

['机动车类型', '术语和定义', '-0.178000', '-0.075456', '-0.102391', '-0.342397', '0.067747', '-0.242497', '0.096015', '0.453227', '0.037681', '0.050730', '-0.078927', '0.241917', '0.209274', '-0.120945', '0.241699', '-0.283224', '0.239738', '-0.092616', '-0.012307', '-0.053337', '-0.388031', '-0.315847', '-0.085938', '0.130480', '0.423218', '0.039561', '0.015704', '0.326262', '0.081554', '0.113729', '-0.081734', '-0.069344', '0.393993', '-0.325369', '-0.306744', '-0.396619', '0.253518', '0.120623', '-0.047676', '-0.004955', '0.472060', '-0.026682', '0.947739', '0.220193', '-0.002648', '0.523070', '-0.417827', '-0.364690', '0.467112', '0.144276', '-0.429806', '-0.354856', '-0.440021', '0.189851', '0.158961', '-0.280760', '-0.291002', '0.279394', '-0.091795', '-0.102762', '0.322889', '0.011151', '0.200561', '0.370954', '-0.147812', '-0.356805', '-0.177177', '0.365495', '0.296862', '0.473389', '-0.162588', '0.245579', '0.089292', '-0.020059', '-0.179904', '0.211901', '0.318997', '-0.022589', '0.059

['陈光蕊赴任逢灾', '江流僧复仇报本', '0.344838', '0.181288', '0.472039', '0.456241', '0.296229', '0.088382', '0.181353', '0.378867', '-0.168174', '-0.005574', '-0.045794', '-0.056061', '0.072943', '-0.301995', '0.117513', '-0.447865', '0.059259', '0.460841', '-0.034492', '-0.021669', '-0.215963', '0.051875', '-0.068262', '0.223965', '-0.023156', '0.284812', '-0.170479', '0.003787', '0.142052', '-0.121413', '-0.073166', '-0.041838', '-0.101860', '0.095027', '-0.130391', '-0.099460', '0.188845', '0.036163', '0.049436', '0.256083', '0.067856', '0.300597', '0.384203', '0.143471', '-0.288999', '-0.229220', '-0.172652', '-0.248755', '0.140185', '0.184296', '0.563174', '0.383811', '-0.133446', '-0.221775', '0.121340', '-0.176909', '0.131688', '0.666276', '-0.227184', '0.276687', '0.122244', '0.041620', '0.239657', '0.176043', '0.036191', '0.071421', '0.034100', '0.257724', '0.083032', '0.345439', '-0.547538', '-0.180038', '0.282708', '0.341907', '0.238617', '0.629855', '0.020074', '-0.112642', '0.041982', 

['九九数完魔灭尽', '三三行满道归根', '0.070571', '0.126631', '0.292493', '0.286409', '0.081038', '-0.059735', '0.382957', '0.254528', '-0.177775', '-0.067119', '0.001311', '-0.276439', '0.157709', '-0.181532', '0.437621', '-0.152220', '0.075323', '0.216007', '-0.142253', '-0.054353', '0.054397', '0.200583', '0.130856', '0.156137', '-0.015650', '0.274447', '-0.008305', '0.057524', '0.209333', '0.038609', '-0.255876', '-0.124360', '-0.178314', '0.100502', '-0.158351', '0.090410', '0.171713', '0.156794', '-0.263908', '0.220089', '0.258414', '0.499164', '0.399830', '-0.107864', '-0.189891', '0.051966', '-0.108056', '-0.133518', '0.330954', '-0.005026', '0.248205', '0.183282', '-0.030517', '0.152493', '-0.074707', '-0.135904', '0.237053', '0.588588', '0.035815', '0.093265', '0.116842', '0.138388', '0.125228', '0.284039', '-0.170458', '0.004019', '-0.141621', '0.342976', '0.027560', '0.170843', '-0.251009', '0.051984', '0.368858', '0.137810', '0.444054', '0.550235', '0.026893', '-0.230573', '-0.027204', '

8824320

In [10]:
#index对应到vector 建立embedding_matrix
EMBEDDING_DIM = 200
embedding_matrix = np.zeros((len(word_index) , EMBEDDING_DIM))  # 初始化一个权重向量
for word, i in word_index.items():
    #print(word,i)
    embedding_vector = embeddings_index.get(word) # 取一个word在表中查找是否有这个词
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector #如果有就保存下来
print(embedding_matrix.shape)
# 保存 embedding_matrix
np.save("embedding_matrix_td-lstm.npy",embedding_matrix) 

(21989, 200)


In [9]:
# 读取 embedding_matrix
embedding_matrix = np.load("embedding_matrix_td-lstm.npy")

In [10]:
# 将类别变成one-hot编码
from tensorflow.keras.utils import to_categorical
label_list=to_categorical(np.array(train_data['label'].values))

# 分割数据集
features_left = features[:train_data.shape[0]] # 左输入
features_right = features[train_data.shape[0]:] # 右输入

split_idx = 6000
train_x_left, remaining_x_left = features_left[:split_idx], features_left[split_idx:]
train_x_right, remaining_x_right = features_right[:split_idx], features_right[split_idx:]

train_y, remaining_y = label_list[:split_idx], label_list[split_idx:]

test_idx = 1240
val_x_left, test_x_left = remaining_x_left[:test_idx], remaining_x_left[test_idx:]
val_x_right, test_x_right = remaining_x_right[:test_idx], remaining_x_right[test_idx:]

val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x_left.shape), 
      "\nValidation set: \t{}".format(val_x_left.shape),
      "\nTest set: \t\t{}".format(test_x_left.shape))

			Feature Shapes:
Train set: 		(6000, 100) 
Validation set: 	(1240, 100) 
Test set: 		(2000, 100)


In [13]:
import numpy
import keras
from keras import backend as K
from keras import activations
from keras.engine.topology import Layer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional,SpatialDropout1D,concatenate
K.clear_session()


# 建立模型
def create_classify_model(max_len, vocab_size, embedding_size, class_nums):
    input_l = Input(shape=(max_len, ))
    input_r = Input(shape=(max_len, ))
    
    word_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size,weights=[embedding_matrix],trainable = True)
    
    input_l_embed = SpatialDropout1D(0.2)(word_embedding(input_l))
    input_r_embed = SpatialDropout1D(0.2)(word_embedding(input_r))

    hidden_l = LSTM(64)(input_l_embed)
    hidden_r = LSTM(64, go_backwards=True)(input_r_embed)

    hidden_concat = concatenate([hidden_l, hidden_r], axis=-1)
    
    outputs = Dense(class_nums, activation='softmax')(hidden_concat)
    
    model = Model([input_l, input_r], outputs)
    model.summary()

    return model


Using TensorFlow backend.


In [14]:
MAX_LEN = 100
VOCAB_SIZE = len(word_index)
EMBEDDING_SIZE = 200


CLASS_NUMS = 3
BATCH_SIZE = 128
EPOCHS = 20

import os
import keras.backend.tensorflow_backend as KTF

#进行配置，每个GPU使用60%上限现存
os.environ["CUDA_VISIBLE_DEVICES"]="2" # 使用编号为1，2号的GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不满显存, 自适应分配
#config.gpu_options.per_process_gpu_memory_fraction = 0.6 # 每个GPU现存上届控制在60%以内
session = tf.Session(config=config)
# 设置session
KTF.set_session(session )

# 指定可见显卡
#config.gpu_options.visible_device_list="1"

In [15]:
# create model
model_lstm = create_classify_model(MAX_LEN, VOCAB_SIZE, EMBEDDING_SIZE,  CLASS_NUMS)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 200)     4397800     input_1[0][0]                    
                                                                 input_2[0][0]                    
___________________________________________________________________________________________

In [16]:
# loss and optimizer
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    verbose=0,
    mode='auto', 
    restore_best_weights=True)
# train model
history = model_lstm.fit([train_x_left,train_x_right], train_y, batch_size=BATCH_SIZE, 
          epochs=EPOCHS, validation_data=([val_x_left,val_x_right], val_y),
        callbacks=[early_stop])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 6000 samples, validate on 1240 samples
Epoch 1/20
6000/6000 [==============================] - 21s 4ms/step - loss: 0.9547 - acc: 0.5420 - val_loss: 0.8076 - val_acc: 0.6565
Epoch 2/20
6000/6000 [==============================] - 20s 3ms/step - loss: 0.7507 - acc: 0.6752 - val_loss: 0.6940 - val_acc: 0.7073
Epoch 3/20
6000/6000 [==============================] - 19s 3ms/step - loss: 0.6198 - acc: 0.7418 - val_loss: 0.6331 - val_acc: 0.7468
Epoch 4/20
6000/6000 [==============================] - 19s 3ms/step - loss: 0.5235 - acc: 0.7947 - val_loss: 0.6034 - val_acc: 0.7540
Epoch 5/20
6000/6000 [==============================] - 19s 3ms/step - loss: 0.4372 - acc: 0.8303 - val_loss: 0.6465 - val_acc: 0.7435
Epoch 6/20
6000/6000 [==============================] - 19s 3ms/step - loss: 0.3547 - acc: 0.8682 - val_loss: 0.6322 - val_acc: 0.7605


In [18]:
# 评估模型
test_loss, test_acc = model_lstm.evaluate([test_x_left,test_x_right], test_y, batch_size=BATCH_SIZE)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

2000/2000 [==============================] - 2s 1ms/step
Test Loss: 0.6187303719520569
Test Accuracy: 0.7435


In [19]:
y_pre = model_lstm.predict([test_x_left,test_x_right],batch_size=1024)
y_pre = y_pre.argmax(1).tolist()
test_y_origin = np.argmax(test_y, axis=1).tolist()

In [20]:
from sklearn.metrics import classification_report

print(classification_report(test_y_origin, y_pre,digits=4))

              precision    recall  f1-score   support

           0     0.6671    0.7324    0.6982       725
           1     0.7518    0.5761    0.6523       368
           2     0.8069    0.8203    0.8136       907

    accuracy                         0.7435      2000
   macro avg     0.7419    0.7096    0.7214      2000
weighted avg     0.7461    0.7435    0.7421      2000



In [61]:
y_pre = model_lstm.predict([test_x_left,test_x_right],batch_size=1024)
y_pre

array([[0.9802361 , 0.01016279, 0.00960107],
       [0.16952749, 0.7307094 , 0.09976318],
       [0.7674025 , 0.02266994, 0.20992756],
       ...,
       [0.23051888, 0.2748227 , 0.49465844],
       [0.29850417, 0.6603089 , 0.04118686],
       [0.10908933, 0.00636397, 0.88454664]], dtype=float32)

In [62]:
np.save("pre_td_sltm.npy",y_pre) 

In [63]:
# 保存RNN模型
model_lstm.save('model_lstm_attention.h5')